In [1]:
### DEFINITIONS ###
import harp
import pandas as pd 

FOLDER = "olfactometer_13A_2025-11-06T062858Z"
PATH = "C:/Data/Olfact/" + FOLDER  

# Containers
RegList = {}

In [2]:
### OLFACTOMETER REGISTERS LIST ###
import harp  
# Create reader for olfactometer data
OlfReader = harp.create_reader(PATH + "/behavior/Olfactometer.harp", include_common_registers=False)

print(list(OlfReader.registers.keys())) # List of all olfactometer registers

['WhoAmI', 'HardwareVersionHigh', 'HardwareVersionLow', 'AssemblyVersion', 'CoreVersionHigh', 'CoreVersionLow', 'FirmwareVersionHigh', 'FirmwareVersionLow', 'TimestampSeconds', 'TimestampMicroseconds', 'OperationControl', 'ResetDevice', 'DeviceName', 'SerialNumber', 'ClockConfiguration', 'EnableFlow', 'Flowmeter', 'DI0State', 'Channel0UserCalibration', 'Channel1UserCalibration', 'Channel2UserCalibration', 'Channel3UserCalibration', 'Channel4UserCalibration', 'Channel3UserCalibrationAux', 'UserCalibrationEnable', 'Channel0TargetFlow', 'Channel1TargetFlow', 'Channel2TargetFlow', 'Channel3TargetFlow', 'Channel4TargetFlow', 'ChannelsTargetFlow', 'Channel0ActualFlow', 'Channel1ActualFlow', 'Channel2ActualFlow', 'Channel3ActualFlow', 'Channel4ActualFlow', 'Channel0Frequency', 'Channel1Frequency', 'Channel2Frequency', 'Channel3Frequency', 'Channel4Frequency', 'Channel0DutyCycle', 'Channel1DutyCycle', 'Channel2DutyCycle', 'Channel3DutyCycle', 'Channel4DutyCycle', 'DigitalOutputSet', 'DigitalOu

In [3]:
### OLFACTOMETER - ALL DATA TO CSV ###

# Create reader for olfactometer data
OlfReader = harp.create_reader(PATH + "/behavior/Olfactometer.harp", include_common_registers=False)

# Containers
metadata_rows = {}
alldf = pd.DataFrame()

# Loop through all olfactometer registers
for name, reg in OlfReader.registers.items():
    try:
        df = reg.read()
        df.index = df.index.round(3)

        # Rename columns only if they contain "Channel" Eg: ['Flowmeter', 'Channel1', 'Channel2'] will be ['Flowmeter', 'Flowmeter-Ch1', 'Flowmeter-Ch2']
        new_cols = []
        for col in df.columns:
            if "channel" in col.lower():
                new_cols.append(f"{name}-Ch{col[7:]}")
            else:
                new_cols.append(col)
        df.columns = new_cols

        # If the register has less than 5 rows, treat it as metadata
        if len(df) < 5:
            metadata_rows[name] = df.iloc[0].to_dict()
        else:
            alldf = pd.concat([alldf, df], axis=1)


    except Exception as e:
        print(f"Skipping register {name} {df.head()}: {e}")

# Create metadata DataFrame
if metadata_rows:
    metadata_df = pd.DataFrame.from_dict(metadata_rows, orient='index')
    print("\n📋 Metadata DataFrame:")
    print(metadata_df)
else:
    print("⚠️ No metadata rows found.")

alldf.index = (alldf.index - alldf.index[0])
# Save merged DataFrame
alldf.to_csv('merged_df.csv', index=True)


Skipping register EndValveState              EndValve0  EndValve1  ValveDummy
Time                                         
4693311.086      False      False       False
4693311.123      False      False       False
4693321.130       True      False       False
4693323.135      False      False       False
4693328.147       True      False       False: Reindexing only valid with uniquely valued Index objects

📋 Metadata DataFrame:
                              WhoAmI  HardwareVersionHigh  HardwareVersionLow  \
WhoAmI                        1140.0                  NaN                 NaN   
HardwareVersionHigh              NaN                  1.0                 NaN   
HardwareVersionLow               NaN                  NaN                 1.0   
AssemblyVersion                  NaN                  NaN                 NaN   
CoreVersionHigh                  NaN                  NaN                 NaN   
...                              ...                  ...                 ...  